In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import pyshark
import threading
from collections import deque
import time

# Enhanced attack type mapping based on KDD Cup 1999 patterns
def enhance_attack_classification(df):
    """
    Map anomalies to specific attack types based on feature patterns
    """
    def classify_attack_type(row):
        if row['class'] == 'normal':
            return 'normal'

        # DOS Attacks - high volume, many connections
        if (row['count'] > 100 and
            row['duration'] == 0 and
            row['src_bytes'] < 100):
            return 'dos'

        # Probe Attacks - scanning activities
        elif (row['num_failed_logins'] > 0 or
              row['wrong_fragment'] > 0 or
              row['num_compromised'] > 0):
            return 'probe'

        # U2R Attacks - privilege escalation
        elif (row['num_root'] > 0 or
              row['num_file_creations'] > 0 or
              row['num_shells'] > 0):
            return 'u2r'

        # U2L Attacks - unauthorized access
        elif (row['num_access_files'] > 0 or
              row['num_outbound_cmds'] > 0 or
              row['is_guest_login'] > 0):
            return 'u2l'

        else:
            return 'unknown_attack'

    df['attack_type'] = df.apply(classify_attack_type, axis=1)
    return df

# Load and enhance dataset
def load_enhanced_dataset(file_path):
    df = pd.read_csv(file_path)

    # Convert categorical features
    categorical_columns = ['protocol_type', 'service', 'flag']
    label_encoders = {}

    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

    # Enhanced attack classification
    df = enhance_attack_classification(df)

    # Encode attack types
    le_attack = LabelEncoder()
    df['attack_type_encoded'] = le_attack.fit_transform(df['attack_type'])
    label_encoders['attack_type'] = le_attack

    print("Enhanced Attack Type Distribution:")
    print(df['attack_type'].value_counts())

    return df, label_encoders

In [ ]:
class DeepLearningNIDS:
    def __init__(self, sequence_length=10, n_features=41):
        self.sequence_length = sequence_length
        self.n_features = n_features
        self.models = {}

    def create_lstm_model(self):
        """LSTM model for temporal pattern recognition"""
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import LSTM, Dense, Dropout
        from tensorflow.keras.optimizers import Adam

        model = Sequential([
            LSTM(64, return_sequences=True, input_shape=(self.sequence_length, self.n_features)),
            Dropout(0.2),
            LSTM(32, return_sequences=False),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(6, activation='softmax')  # Changed to 6 classes
        ])

        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def create_cnn_model(self):
        """CNN model for spatial pattern recognition"""
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
        from tensorflow.keras.optimizers import Adam

        model = Sequential([
            Conv1D(64, 3, activation='relu', input_shape=(self.sequence_length, self.n_features)),
            MaxPooling1D(2),
            Conv1D(32, 3, activation='relu'),
            MaxPooling1D(2),
            Flatten(),
            Dense(64, activation='relu'),
            Dropout(0.3),
            Dense(32, activation='relu'),
            Dense(6, activation='softmax')  # Changed to 6 classes
        ])

        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def create_hybrid_model(self):
        """Hybrid CNN-LSTM model"""
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
        from tensorflow.keras.optimizers import Adam

        model = Sequential([
            Conv1D(64, 3, activation='relu', input_shape=(self.sequence_length, self.n_features)),
            MaxPooling1D(2),
            Conv1D(32, 3, activation='relu'),
            LSTM(32, return_sequences=False),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(6, activation='softmax')  # Changed to 6 classes
        ])

        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def prepare_sequences(self, X, y, sequence_length=10):
        """Convert data to sequences for deep learning"""
        X_sequences, y_sequences = [], []

        for i in range(len(X) - sequence_length):
            X_sequences.append(X[i:(i + sequence_length)])
            y_sequences.append(y[i + sequence_length - 1])

        return np.array(X_sequences), np.array(y_sequences)

    def train_models(self, X_train, y_train, X_val, y_val, epochs=50):
        """Train all deep learning models"""
        # Prepare sequential data
        X_seq_train, y_seq_train = self.prepare_sequences(X_train, y_train, self.sequence_length)
        X_seq_val, y_seq_val = self.prepare_sequences(X_val, y_val, self.sequence_length)

        models = {
            'LSTM': self.create_lstm_model(),
            'CNN': self.create_cnn_model(),
            'Hybrid_CNN_LSTM': self.create_hybrid_model()
        }

        history = {}
        for name, model in models.items():
            print(f"Training {name} model...")
            history[name] = model.fit(
                X_seq_train, y_seq_train,
                validation_data=(X_seq_val, y_seq_val),
                epochs=epochs,
                batch_size=32,
                verbose=1
            )
            self.models[name] = model

        return history

In [ ]:
class RealTimeNIDS:
    def __init__(self, ml_models, scaler, label_encoders, interface='eth0'):
        self.ml_models = ml_models
        self.scaler = scaler
        self.label_encoders = label_encoders
        self.interface = interface
        self.packet_buffer = deque(maxlen=1000)
        self.feature_buffer = deque(maxlen=50)
        self.is_monitoring = False
        self.attack_count = 0
        self.normal_count = 0

    def extract_features_from_packet(self, packet):
        """Extract NIDS features from network packet"""
        try:
            features = {
                'duration': 0,
                'protocol_type': packet.transport_layer.lower() if hasattr(packet, 'transport_layer') else 'tcp',
                'service': packet.dst_port if hasattr(packet, 'dst_port') else '0',
                'flag': 'SF',  # Simplified
                'src_bytes': len(packet) if hasattr(packet, '__len__') else 0,
                'dst_bytes': 0,
                'land': 0,
                'wrong_fragment': 0,
                'urgent': 0,
                'hot': 0,
                'num_failed_logins': 0,
                'logged_in': 1 if packet.dst_port in [80, 443, 22] else 0,
                'num_compromised': 0,
                'root_shell': 0,
                'su_attempted': 0,
                'num_root': 0,
                'num_file_creations': 0,
                'num_shells': 0,
                'num_access_files': 0,
                'num_outbound_cmds': 0,
                'is_host_login': 0,
                'is_guest_login': 0,
                'count': 1,
                'srv_count': 1,
                'serror_rate': 0,
                'srv_serror_rate': 0,
                'rerror_rate': 0,
                'srv_rerror_rate': 0,
                'same_srv_rate': 1.0,
                'diff_srv_rate': 0.0,
                'srv_diff_host_rate': 0.0,
                'dst_host_count': 1,
                'dst_host_srv_count': 1,
                'dst_host_same_srv_rate': 1.0,
                'dst_host_diff_srv_rate': 0.0,
                'dst_host_same_src_port_rate': 0.0,
                'dst_host_srv_diff_host_rate': 0.0,
                'dst_host_serror_rate': 0.0,
                'dst_host_srv_serror_rate': 0.0,
                'dst_host_rerror_rate': 0.0,
                'dst_host_srv_rerror_rate': 0.0
            }
            return features
        except Exception as e:
            print(f"Error extracting features: {e}")
            return None

    def packet_handler(self, packet):
        """Handle incoming packets in real-time"""
        if self.is_monitoring:
            features = self.extract_features_from_packet(packet)
            if features:
                self.packet_buffer.append(features)
                self.analyze_packet(features)

    def analyze_packet(self, features):
        """Analyze packet using ML models"""
        # Convert to DataFrame
        df_features = pd.DataFrame([features])

        # Preprocess
        categorical_columns = ['protocol_type', 'service', 'flag']
        for col in categorical_columns:
            if col in df_features.columns:
                le = self.label_encoders.get(col)
                if le:
                    # Handle unseen labels
                    df_features[col] = df_features[col].apply(
                        lambda x: x if x in le.classes_ else 'unknown'
                    )
                    try:
                        df_features[col] = le.transform(df_features[col])
                    except:
                        df_features[col] = 0

        # Feature engineering
        df_features['bytes_ratio'] = df_features['src_bytes'] / (df_features['dst_bytes'] + 1)
        df_features['total_bytes'] = df_features['src_bytes'] + df_features['dst_bytes']
        df_features['connection_speed'] = df_features['total_bytes'] / (df_features['duration'] + 1)

        # Scale features
        try:
            features_scaled = self.scaler.transform(df_features)

            # Predict using all models
            for model_name, model in self.ml_models.items():
                if hasattr(model, 'predict'):
                    prediction = model.predict(features_scaled)
                    prediction_proba = model.predict_proba(features_scaled)

                    predicted_label = self.label_encoders['attack_type'].inverse_transform(prediction)
                    confidence = np.max(prediction_proba) * 100

                    if predicted_label[0] != 'normal' and confidence > 80:
                        self.attack_count += 1
                        print(f"🚨 ALERT: {model_name} detected {predicted_label[0]} attack! "
                              f"Confidence: {confidence:.2f}%")
                        self.log_attack(features, predicted_label[0], confidence, model_name)
                    else:
                        self.normal_count += 1

        except Exception as e:
            print(f"Prediction error: {e}")

    def log_attack(self, features, attack_type, confidence, model_name):
        """Log detected attacks"""
        timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
        log_entry = {
            'timestamp': timestamp,
            'attack_type': attack_type,
            'confidence': confidence,
            'model': model_name,
            'src_bytes': features.get('src_bytes', 0),
            'protocol': features.get('protocol_type', 'unknown')
        }

        # Save to log file
        with open('attack_log.csv', 'a') as f:
            f.write(f"{timestamp},{attack_type},{confidence},{model_name},{features.get('src_bytes', 0)},{features.get('protocol_type', 'unknown')}\n")

        print(f"📝 Logged: {timestamp} - {attack_type} (Confidence: {confidence:.1f}%)")

    def start_monitoring(self, duration=300):
        """Start real-time network monitoring"""
        print(f"🎯 Starting real-time NIDS monitoring on interface {self.interface}...")
        self.is_monitoring = True

        try:
            # Use pyShark for packet capture
            capture = pyshark.LiveCapture(interface=self.interface)
            capture.apply_on_packets(self.packet_handler, timeout=duration)

        except Exception as e:
            print(f"Monitoring error: {e}")

        finally:
            self.stop_monitoring()

    def stop_monitoring(self):
        """Stop monitoring and generate report"""
        self.is_monitoring = False
        print("\n📊 Monitoring Session Summary:")
        print(f"Normal packets: {self.normal_count}")
        print(f"Attack packets: {self.attack_count}")
        print(f"Total packets analyzed: {self.normal_count + self.attack_count}")

        if self.attack_count > 0:
            print("🚨 SECURITY ALERT: Potential attacks detected!")

In [ ]:
class UnsupervisedAnomalyDetector:
    def __init__(self, contamination=0.1):
        self.isolation_forest = IsolationForest(
            contamination=contamination,
            random_state=42
        )
        self.dbscan = DBSCAN(eps=0.5, min_samples=10)
        self.is_trained = False

    def train_unsupervised_models(self, X):
        """Train unsupervised models for anomaly detection"""
        print("Training unsupervised anomaly detection models...")

        # Isolation Forest
        self.isolation_forest.fit(X)

        # DBSCAN for clustering
        # Note: DBSCAN doesn't need explicit training

        self.is_trained = True
        print("Unsupervised models trained successfully!")

    def detect_anomalies(self, X):
        """Detect anomalies using multiple unsupervised methods"""
        if not self.is_trained:
            raise ValueError("Models not trained yet!")

        results = {}

        # Isolation Forest predictions (-1 for anomalies, 1 for normal)
        if_scores = self.isolation_forest.decision_function(X)
        if_predictions = self.isolation_forest.predict(X)

        results['isolation_forest'] = {
            'predictions': if_predictions,
            'scores': if_scores,
            'anomalies': if_predictions == -1
        }

        # DBSCAN clustering (-1 for noise/anomalies)
        dbscan_labels = self.dbscan.fit_predict(X)
        results['dbscan'] = {
            'labels': dbscan_labels,
            'anomalies': dbscan_labels == -1
        }

        # Combined anomaly detection
        combined_anomalies = (if_predictions == -1) | (dbscan_labels == -1)
        results['combined'] = {
            'anomalies': combined_anomalies,
            'confidence': np.abs(if_scores)  # Use absolute scores as confidence
        }

        return results

    def explain_anomalies(self, X, feature_names, results):
        """Provide explanations for detected anomalies"""
        anomaly_indices = np.where(results['combined']['anomalies'])[0]

        explanations = []
        for idx in anomaly_indices[:5]:  # Explain first 5 anomalies
            explanation = {
                'index': idx,
                'isolation_forest_score': results['isolation_forest']['scores'][idx],
                'dbscan_cluster': results['dbscan']['labels'][idx],
                'top_features': self.get_important_features(X[idx], feature_names)
            }
            explanations.append(explanation)

        return explanations

    def get_important_features(self, sample, feature_names, top_n=3):
        """Get most important features contributing to anomaly"""
        # For simplicity, use feature values with highest magnitude
        feature_importance = np.abs(sample)
        top_indices = np.argsort(feature_importance)[-top_n:][::-1]

        return [(feature_names[i], sample[i]) for i in top_indices]

In [ ]:
class CompleteNIDS:
    def __init__(self):
        self.df = None
        self.label_encoders = None
        self.scaler = StandardScaler()
        self.dl_nids = DeepLearningNIDS()
        self.unsupervised_detector = UnsupervisedAnomalyDetector()
        self.real_time_nids = None
        self.models = {}

    def run_complete_pipeline(self, file_path):
        """Run complete NIDS pipeline"""
        print("🚀 Starting Complete Network Intrusion Detection System")
        print("=" * 60)

        # 1. Load and enhance dataset
        self.df, self.label_encoders = load_enhanced_dataset(file_path)

        # 2. Prepare features
        X = self.df.drop(['class', 'attack_type', 'attack_type_encoded'], axis=1)
        y = self.df['attack_type_encoded']

        # Split data
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42
        )

        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        # 3. Train traditional ML models
        from sklearn.ensemble import RandomForestClassifier
        from xgboost import XGBClassifier

        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(X_train_scaled, y_train)
        self.models['Random_Forest'] = rf_model

        xgb_model = XGBClassifier(random_state=42)
        xgb_model.fit(X_train_scaled, y_train)
        self.models['XGBoost'] = xgb_model

        # 4. Train deep learning models
        print("\n🧠 Training Deep Learning Models...")
        X_val, X_test_dl, y_val, y_test_dl = train_test_split(
            X_test_scaled, y_test, test_size=0.5, random_state=42
        )

        # Convert y to numpy arrays for sequence preparation
        dl_history = self.dl_nids.train_models(
            X_train_scaled, y_train.values, X_val, y_val.values, epochs=30
        )

        # 5. Train unsupervised models
        print("\n🔍 Training Unsupervised Anomaly Detection...")
        self.unsupervised_detector.train_unsupervised_models(X_train_scaled)

        # 6. Initialize real-time monitoring
        self.real_time_nids = RealTimeNIDS(
            self.models, self.scaler, self.label_encoders
        )

        # 7. Evaluate all models
        self.evaluate_all_models(X_test_scaled, y_test)

        print("\n✅ NIDS System Ready!")
        return self

    def evaluate_all_models(self, X_test, y_test):
        """Comprehensive evaluation of all models"""
        from sklearn.metrics import classification_report, accuracy_score

        print("\n📊 Comprehensive Model Evaluation:")
        print("=" * 50)

        # Traditional ML models
        for name, model in self.models.items():
            if hasattr(model, 'predict'):
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                print(f"{name} Accuracy: {accuracy:.4f}")

        # Unsupervised detection evaluation
        unsupervised_results = self.unsupervised_detector.detect_anomalies(X_test)
        anomaly_rate = np.mean(unsupervised_results['combined']['anomalies'])
        print(f"Unsupervised Anomaly Detection Rate: {anomaly_rate:.4f}")

        # Test real-time detection with sample
        print("\n🔍 Testing Real-time Detection...")
        sample_features = self.df.drop(['class', 'attack_type', 'attack_type_encoded'], axis=1).iloc[:1]
        self.real_time_nids.analyze_packet(sample_features.iloc[0].to_dict())

    def start_real_time_monitoring(self, duration=60):
        """Start real-time network monitoring"""
        if self.real_time_nids:
            print(f"\n🎯 Starting {duration}-second real-time monitoring...")
            self.real_time_nids.start_monitoring(duration)
        else:
            print("❌ Real-time NIDS not initialized!")

    def detect_zero_day_attacks(self, new_data):
        """Detect potential zero-day attacks using unsupervised learning"""
        print("\n🕵️ Detecting Zero-Day Attacks...")

        if isinstance(new_data, pd.DataFrame):
            new_data_scaled = self.scaler.transform(new_data)
        else:
            new_data_scaled = self.scaler.transform([new_data])

        results = self.unsupervised_detector.detect_anomalies(new_data_scaled)

        anomaly_count = np.sum(results['combined']['anomalies'])
        if anomaly_count > 0:
            print(f"🚨 POTENTIAL ZERO-DAY ATTACKS DETECTED: {anomaly_count} anomalies found!")

            # Get explanations
            feature_names = [f'feature_{i}' for i in range(new_data_scaled.shape[1])]
            explanations = self.unsupervised_detector.explain_anomalies(
                new_data_scaled, feature_names, results
            )

            for exp in explanations:
                print(f"Anomaly {exp['index']}: Score={exp['isolation_forest_score']:.3f}")
                print(f"  Suspicious features: {exp['top_features']}")
        else:
            print("✅ No zero-day attacks detected.")

        return results

# Main execution
if __name__ == "__main__":
    # Initialize and run complete NIDS
    nids_system = CompleteNIDS()
    nids_system.run_complete_pipeline('Train_data.csv')

    # Test with sample data
    sample_data = nids_system.df.drop(['class', 'attack_type', 'attack_type_encoded'], axis=1).iloc[100:105]

    # Detect zero-day attacks
    nids_system.detect_zero_day_attacks(sample_data)

    # Uncomment to start real-time monitoring (requires appropriate permissions)
    # nids_system.start_real_time_monitoring(duration=30)

🚀 Starting Complete Network Intrusion Detection System
Enhanced Attack Type Distribution:
attack_type
normal            13449
dos                7321
unknown_attack     3960
probe               399
u2l                  59
u2r                   4
Name: count, dtype: int64

🧠 Training Deep Learning Models...
Training LSTM model...
Epoch 1/30
551/551 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.7485 - loss: 0.7527 - val_accuracy: 0.9400 - val_loss: 0.1861
Epoch 2/30
551/551 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.9401 - loss: 0.2029 - val_accuracy: 0.9560 - val_loss: 0.1437
Epoch 3/30
551/551 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.9533 - loss: 0.1665 - val_accuracy: 0.9570 - val_loss: 0.1275
Epoch 4/30
551/551 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.9571 - loss: 0.1306 - val_accuracy: 0.9626 - val_loss: 0.0930
Epoch 5/30
551/551 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.9705 - loss: 0.0904 - val_accuracy: 0.9732 - val_loss: 0.0714
Epoch 6/30
551/551 ━━━

In [ ]:
# test_nids.py
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification

def create_test_dataset():
    """Create a synthetic test dataset since we don't have the original Train_data.csv"""

    # Create synthetic features matching KDD Cup 1999 format
    n_samples = 1000
    n_features = 41

    # Generate synthetic data
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=20,
        n_redundant=10,
        n_clusters_per_class=1,
        n_classes=5,  # 5 attack types + normal
        random_state=42
    )

    # Create DataFrame with KDD-like feature names
    feature_names = [
        'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
        'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
        'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
        'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds',
        'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
        'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
        'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
        'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
        'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
        'dst_host_rerror_rate', 'dst_host_srv_rerror_rate'
    ]

    df = pd.DataFrame(X, columns=feature_names)

    # Add categorical columns (protocol_type, service, flag)
    df['protocol_type'] = np.random.choice([0, 1, 2], n_samples)  # tcp, udp, icmp
    df['service'] = np.random.choice(range(10), n_samples)  # 10 service types
    df['flag'] = np.random.choice(range(8), n_samples)  # 8 flag types

    # Add class labels
    attack_types = ['normal', 'dos', 'probe', 'u2r', 'u2l', 'unknown_attack']
    df['class'] = [attack_types[i % 6] for i in y]

    # Save test dataset
    df.to_csv('test_data.csv', index=False)
    print(f"Created test dataset with {n_samples} samples")

    return df

# Create test data
test_df = create_test_dataset()
print("Test dataset created successfully!")
print(test_df['class'].value_counts())

Created test dataset with 1000 samples
Test dataset created successfully!
class
u2l       202
u2r       201
normal    201
probe     199
dos       197
Name: count, dtype: int64


In [ ]:
# test_nids_system.py
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Import necessary components (assuming they are defined in the notebook)
# from NIDS_LLM import (
#     CompleteNIDS,
#     load_enhanced_dataset,
#     DeepLearningNIDS,
#     UnsupervisedAnomalyDetector,
#     enhance_attack_classification
# )

def test_nids_system():
    """Test the complete NIDS system"""

    # Check if test data exists, create if not
    if not os.path.exists('test_data.csv'):
        print("Creating test dataset...")
        create_test_dataset() # Assuming create_test_dataset is defined in the notebook

    try:
        # Initialize the system
        print("🚀 Initializing NIDS System...")
        nids_system = CompleteNIDS() # Use the class directly

        # Run pipeline with test data
        print("📊 Running Complete Pipeline...")
        nids_system.run_complete_pipeline('test_data.csv')

        print("✅ Pipeline completed successfully!")

        # Test zero-day attack detection
        print("\n🕵️ Testing Zero-Day Attack Detection...")
        sample_data = nids_system.df.drop(['class', 'attack_type', 'attack_type_encoded'], axis=1).iloc[100:105]
        nids_system.detect_zero_day_attacks(sample_data)

        # Test model saving
        print("\n💾 Testing Model Saving...")
        try:
            import joblib
            joblib.dump(nids_system.models['Random_Forest'], 'nids_model.pkl')
            print("✅ Model saved successfully!")

            # Test model loading
            loaded_model = joblib.load('nids_model.pkl')
            print("✅ Model loaded successfully!")
        except Exception as e:
            print(f"❌ Model saving/loading test failed: {e}")

        return nids_system

    except Exception as e:
        print(f"❌ NIDS System test failed: {e}")
        import traceback
        traceback.print_exc()
        return None

def test_individual_components():
    """Test individual components of the NIDS system"""

    # Access components directly from the notebook's global scope
    # from NIDS_LLM import (
    #     load_enhanced_dataset,
    #     DeepLearningNIDS,
    #     UnsupervisedAnomalyDetector,
    #     enhance_attack_classification
    # )

    print("\n🧪 Testing Individual Components...")

    try:
        # Test dataset loading
        print("1. Testing dataset loading...")
        df, label_encoders = load_enhanced_dataset('test_data.csv') # Use the function directly
        print(f"✅ Dataset loaded: {df.shape}")

        # Test attack classification
        print("2. Testing attack classification...")
        enhanced_df = enhance_attack_classification(df) # Use the function directly
        print(f"✅ Attack classification: {enhanced_df['attack_type'].value_counts()}")

        # Test deep learning model creation
        print("3. Testing DL model creation...")
        dl_nids = DeepLearningNIDS(sequence_length=10, n_features=41) # Use the class directly

        # Test model architectures
        lstm_model = dl_nids.create_lstm_model()
        cnn_model = dl_nids.create_cnn_model()
        hybrid_model = dl_nids.create_hybrid_model()

        print("✅ LSTM model created")
        print("✅ CNN model created")
        print("✅ Hybrid model created")

        # Test unsupervised detector
        print("4. Testing unsupervised detector...")
        unsupervised_detector = UnsupervisedAnomalyDetector(contamination=0.1) # Use the class directly

        # Prepare data for training
        X = df.drop(['class', 'attack_type'], axis=1, errors='ignore')
        if 'attack_type_encoded' in X.columns:
            X = X.drop('attack_type_encoded', axis=1)

        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Train unsupervised models
        unsupervised_detector.train_unsupervised_models(X_scaled)

        # Test anomaly detection
        results = unsupervised_detector.detect_anomalies(X_scaled[:100])
        anomaly_count = np.sum(results['combined']['anomalies'])
        print(f"✅ Unsupervised detection: {anomaly_count} anomalies found in 100 samples")

        print("🎉 All component tests passed!")

    except Exception as e:
        print(f"❌ Component test failed: {e}")
        import traceback
        traceback.print_exc()

def performance_test(nids_system):
    """Test system performance"""

    if nids_system is None:
        print("❌ Cannot run performance test - NIDS system not available")
        return

    print("\n⚡ Running Performance Tests...")

    import time
    from sklearn.metrics import accuracy_score

    try:
        # Prepare test data
        X_test = nids_system.df.drop(['class', 'attack_type', 'attack_type_encoded'], axis=1)
        y_test = nids_system.df['attack_type_encoded']

        # Test prediction speed
        start_time = time.time()

        # Test Random Forest
        rf_predictions = nids_system.models['Random_Forest'].predict(X_test.iloc[:100])
        rf_time = time.time() - start_time

        # Test XGBoost
        start_time = time.time()
        xgb_predictions = nids_system.models['XGBoost'].predict(X_test.iloc[:100])
        xgb_time = time.time() - start_time

        # Calculate accuracy
        rf_accuracy = accuracy_score(y_test.iloc[:100], rf_predictions)
        xgb_accuracy = accuracy_score(y_test.iloc[:100], xgb_predictions)

        print(f"📊 Random Forest - Time: {rf_time:.4f}s, Accuracy: {rf_accuracy:.4f}")
        print(f"📊 XGBoost - Time: {xgb_time:.4f}s, Accuracy: {xgb_accuracy:.4f}")

        # Test unsupervised detection performance
        start_time = time.time()
        unsupervised_results = nids_system.unsupervised_detector.detect_anomalies(X_test.iloc[:100])
        unsupervised_time = time.time() - start_time

        anomaly_rate = np.mean(unsupervised_results['combined']['anomalies'])
        print(f"📊 Unsupervised Detection - Time: {unsupervised_time:.4f}s, Anomaly Rate: {anomaly_rate:.4f}")

    except Exception as e:
        print(f"❌ Performance test failed: {e}")

if __name__ == "__main__":
    print("Starting Comprehensive NIDS System Tests...")
    print("=" * 50)

    # Test individual components first
    test_individual_components()

    # Test complete system
    nids_system = test_nids_system()

    # Run performance tests if system is available
    if nids_system:
        performance_test(nids_system)

    print("\n🎯 Testing Complete!")

Starting Comprehensive NIDS System Tests...

🧪 Testing Individual Components...
1. Testing dataset loading...
Enhanced Attack Type Distribution:
attack_type
probe             715
normal            201
u2r                73
u2l                 7
unknown_attack      4
Name: count, dtype: int64
✅ Dataset loaded: (1000, 44)
2. Testing attack classification...
✅ Attack classification: attack_type
probe             715
normal            201
u2r                73
u2l                 7
unknown_attack      4
Name: count, dtype: int64
3. Testing DL model creation...
✅ LSTM model created
✅ CNN model created
✅ Hybrid model created
4. Testing unsupervised detector...
Training unsupervised anomaly detection models...
Unsupervised models trained successfully!
✅ Unsupervised detection: 100 anomalies found in 100 samples
🎉 All component tests passed!
🚀 Initializing NIDS System...
📊 Running Complete Pipeline...
🚀 Starting Complete Network Intrusion Detection System
Enhanced Attack Type Distribution:
att

In [ ]:
# quick_test.py
import pandas as pd
import numpy as np

def quick_validation():
    """Quick validation of the main components"""

    # Create minimal test data
    data = {
        'duration': [0, 1, 0, 2, 0],
        'protocol_type': ['tcp', 'udp', 'tcp', 'icmp', 'tcp'],
        'service': ['http', 'ftp', 'ssh', 'http', 'dns'],
        'flag': ['SF', 'S0', 'SF', 'REJ', 'SF'],
        'src_bytes': [100, 50, 1000, 200, 150],
        'dst_bytes': [200, 100, 500, 100, 300],
        'land': [0, 0, 0, 0, 0],
        'wrong_fragment': [0, 0, 0, 0, 0],
        'urgent': [0, 0, 0, 0, 0],
        'hot': [0, 0, 0, 0, 0],
        'num_failed_logins': [0, 0, 0, 0, 0],
        'logged_in': [1, 0, 1, 0, 1],
        'num_compromised': [0, 0, 0, 0, 0],
        'root_shell': [0, 0, 0, 0, 0],
        'su_attempted': [0, 0, 0, 0, 0],
        'num_root': [0, 0, 0, 0, 0],
        'num_file_creations': [0, 0, 0, 0, 0],
        'num_shells': [0, 0, 0, 0, 0],
        'num_access_files': [0, 0, 0, 0, 0],
        'num_outbound_cmds': [0, 0, 0, 0, 0],
        'is_host_login': [0, 0, 0, 0, 0],
        'is_guest_login': [0, 0, 0, 0, 0],
        'count': [1, 10, 1, 5, 2],
        'srv_count': [1, 1, 1, 1, 1],
        'serror_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'srv_serror_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'rerror_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'srv_rerror_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'same_srv_rate': [1.0, 0.1, 1.0, 0.2, 1.0],
        'diff_srv_rate': [0.0, 0.9, 0.0, 0.8, 0.0],
        'srv_diff_host_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'dst_host_count': [1, 1, 1, 1, 1],
        'dst_host_srv_count': [1, 1, 1, 1, 1],
        'dst_host_same_srv_rate': [1.0, 0.1, 1.0, 0.2, 1.0],
        'dst_host_diff_srv_rate': [0.0, 0.9, 0.0, 0.8, 0.0],
        'dst_host_same_src_port_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'dst_host_srv_diff_host_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'dst_host_serror_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'dst_host_srv_serror_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'dst_host_rerror_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'dst_host_srv_rerror_rate': [0.0, 0.0, 0.0, 0.0, 0.0],
        'class': ['normal', 'dos', 'normal', 'probe', 'normal']
    }


    df = pd.DataFrame(data)
    df.to_csv('quick_test_data.csv', index=False)

    print("Quick test data created!")
    print(df)

    # Test attack classification
    # from NIDS_LLM import enhance_attack_classification # Removed import

    enhanced_df = enhance_attack_classification(df)
    print("\nEnhanced classification:")
    print(enhanced_df[['class', 'attack_type']])

if __name__ == "__main__":
    quick_validation()

Quick test data created!
   duration protocol_type service flag  src_bytes  dst_bytes  land  \
0         0           tcp    http   SF        100        200     0   
1         1           udp     ftp   S0         50        100     0   
2         0           tcp     ssh   SF       1000        500     0   
3         2          icmp    http  REJ        200        100     0   
4         0           tcp     dns   SF        150        300     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                   1   
1               0       0    0  ...                   1   
2               0       0    0  ...                   1   
3               0       0    0  ...                   1   
4               0       0    0  ...                   1   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                     1.0                     0.0   
1                     0.1                     0.9   
2                     1.0               

In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df_quick_test = pd.read_csv('quick_test_data.csv')

# Display the DataFrame
display(df_quick_test)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,http,SF,100,200,0,0,0,0,...,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal
1,1,udp,ftp,S0,50,100,0,0,0,0,...,1,0.1,0.9,0.0,0.0,0.0,0.0,0.0,0.0,dos
2,0,tcp,ssh,SF,1000,500,0,0,0,0,...,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal
3,2,icmp,http,REJ,200,100,0,0,0,0,...,1,0.2,0.8,0.0,0.0,0.0,0.0,0.0,0.0,probe
4,0,tcp,dns,SF,150,300,0,0,0,0,...,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal


In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df_quick_test = pd.read_csv('test_data.csv')

# Display the DataFrame
display(df_quick_test)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,3.706164,2,6,3,-2.563666,-0.424430,2.903907,3.815563,-1.862075,-0.048614,...,8.157736,-1.087310,-0.056332,-0.468521,4.419663,-7.245419,-4.036911,-1.356477,-2.031775,u2r
1,0.284880,2,0,1,1.259672,-0.988513,3.599875,7.946567,-4.779723,1.218580,...,10.173998,0.311906,-1.280508,2.417896,0.811549,-7.614485,-6.505017,-0.618181,0.572956,u2l
2,1.604088,1,6,7,3.974206,0.321924,-0.577608,6.814004,-12.906638,-0.431139,...,2.031350,-0.056714,0.548524,2.558823,-1.288985,7.358893,0.674510,-0.861171,1.463959,probe
3,2.072260,1,8,6,1.059525,0.027923,2.808889,12.417437,5.987110,-1.783071,...,8.936379,-1.367456,-0.415160,-0.658395,-4.075880,7.788189,-3.017031,-0.716851,3.784893,u2r
4,1.926316,1,7,6,4.763571,-0.866528,-3.465453,-7.070445,-9.440475,-0.136619,...,-5.857448,-0.398494,-0.499035,-4.423595,1.210675,-6.153636,3.217550,0.867847,-1.107657,u2l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.086782,2,6,1,-4.589476,-0.901755,-1.338491,-5.604130,-0.262518,4.289926,...,-5.199086,-0.366586,-0.412953,-4.087142,2.755915,-10.281987,2.369648,-0.097391,-3.951840,u2r
996,-0.103823,1,9,4,1.149824,0.509664,-2.179203,5.152602,2.916554,-5.314440,...,0.247478,1.396360,0.145728,-1.207709,-0.739945,2.483333,-9.848745,2.149942,-2.631047,normal
997,-1.261754,1,1,5,0.608832,0.468626,1.940092,7.042285,-14.772248,1.498921,...,4.593265,1.212831,-1.774526,-0.357779,1.628866,-5.745403,8.145804,0.065375,1.867080,normal
998,-0.021211,2,0,6,-0.890676,-0.874047,-0.226452,10.379560,4.373637,-3.626173,...,2.293858,-1.279738,-2.274869,0.989052,-2.082430,17.850251,-11.093933,-0.384882,1.145201,u2l
